# Beam Steering
Now let's see what happens when we steer the beam to another direction than directly forward (beam_direction = 0). We can steer the beam by adding an additional angle to our steering_vector elements
Also, we have a 4 element array, with a 42mm spacing. So to simplify code, lets set these fixed in our code.

Previously we measured sensitivity by assuming our incoming sound if aimed straight into our array. We used a steering vector to create an array with phase-shifts to add to our microphone channels so that we can 'correct' for incoming delays and used this steering vector to 'probe' our array with all angles we want to measure (-90 .. +90 degrees).

In this example we add another, constant delay to our incoming signal, simulating sound coming from other directions before again probing all angles. Interestingly this is done by another steering_vector: In order to calculate total phase-shift we add the phase-shifts of the incoming sound to the phase-shifts of the probing angle: We can multiply the steering_vector of the beam to the steering_vector of the probing. This is done in the next example and we can now simulate how sensitivity changes as we change our steering angle.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider

%matplotlib widget

# Generate the array geometry (for simplicity, a linear array)
PI = np.pi
Fs = 44100
speed_of_sound = 343.0
default_f0 = 6100
mic_positions = np.array([-1.5, -0.5, 0.5, 1.5, 2.5]) * 0.042

# Construct the steering vector for the given angle
def steering_vector(f0, angle):
    # Create an array with our microphone positions, in meters
    # mic_positions = np.array([n*distance for n in np.arange(num_elements)])
    # Angle to radians
    angle_rad = np.radians(angle)

    # Wavelength
    l = speed_of_sound / f0
    # mic_position in wavelengts
    d = mic_positions / l

    # Create the steering vector with the beam direction applied
    steering_vector = np.exp(-1j * 2*PI*d*np.sin(angle_rad))
    # print(angle_deg, steering_vector, np.sum(steering_vector))

    return steering_vector

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, polar=True)

def plot_sensitivity(f0, beam_direction):
    # Calculate sensitivity for each angle
    probe_angles = np.linspace(-90, 90, 181)  # Angles from -90 to 90 degrees

    steervect = steering_vector(f0, -1*beam_direction)
    sum_vectors = [np.sum(steering_vector(f0, angle) * steervect) for angle in probe_angles]
    sensitivities = np.abs(sum_vectors)

    # We could normalize the sensitivity and use a dB scale.
    dbnorm = lambda x: 20*np.log10(np.abs(x)/np.max(x));
    # sensitivities = dbnorm(sensitivities)
    
    # phases = np.angle(sum_vectors) + PI

    # we could convert to dB scale

    # Plot radial plot
    ax.clear()
    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')
    ax.plot(np.radians(probe_angles), sensitivities)
    # ax.plot(np.radians(angles), phases, '-.')
    ax.set_rlabel_position(0)
    ax.set_thetalim(-1/2*PI, 1/2*PI)

    ax.set_xlabel('Angle (degrees)', fontsize=12, labelpad=15)
    ax.set_ylabel('Sensitivity', fontsize=12, labelpad=15)
    plt.title('Beamformer Sensitivity', fontsize=14, pad=20)
    plt.show()

@interact(f0 = IntSlider(min=100, max=Fs/2, step=1, value = default_f0),
          beam_direction = IntSlider(min=-90, max=90, step=1, value=0)
         )
def plot(f0, beam_direction):
    plot_sensitivity(f0, beam_direction)


interactive(children=(IntSlider(value=6100, description='f0', max=22050, min=100), IntSlider(value=0, descript…